# Tuple

## Architecture

**Important:**
- t_xmin - txid of the transaction that inserted this tuple.
- t_xmax - txid of the transaction that deleted or updated this tuple. If this tuple has not been deleted or updated, t_xmax is set to 0, which means INVALID.
- t_cid - the number of SQL commands that were executed before this command was executed within the current transaction, starting from 0.
- t_ctid - tuple identifier (tid) that points to itself or a new tuple (if there is a new version of the tuple), structure: (page_index, tuple_index_in_page).

**Not In Scope:**
- t_infomask, t_infomask2 - contains information about the tuple that reduce work for processes, in our context hint bits about xmin and xmax are stored here too.
- t_hoff - offset to user data in tuple.

<img src="./helpers/tuple-structure.png" alt="drawing" width="900"/>

## Hands On

Operations:
- INSERT
- UPDATE
- DELETE

```SQL
DROP TABLE IF EXISTS tbl;
SELECT generate_series AS id
INTO tbl
FROM generate_series(1,10);

/*** (detailed in VACUUM module) ***/
-- Turn autovacuum off
ALTER TABLE tbl SET (autovacuum_enabled = false);

-- Look at the page layout
DROP VIEW IF EXISTS vw_inspect_tbl_first_page;
CREATE VIEW vw_inspect_tbl_first_page AS
SELECT
	lp AS line_pointer
	,lp_off AS liner_pointer_offset
	,lp_len AS liner_pointer_length
	,lp_off - lp_len AS next_liner_pointer_offset
	,t_xmin
	,t_xmax
	,t_ctid
FROM 
	heap_page_items(get_raw_page('tbl', 0)) hp;
```SQL


### Update

```SQL
-- Some row
UPDATE tbl SET data = 1 WHERE id = 1;
SELECT * FROM vw_inspect_tbl_first_page;
```

```SQL
-- Same row
UPDATE tbl SET data = 2 WHERE id = 1;
SELECT * FROM vw_inspect_tbl_first_page;
```

## Delete

```SQL
-- Some row
DELETE FROM tbl WHERE id = 2;
SELECT * FROM vw_inspect_tbl_first_page;
```

```SQL
-- Same row
DELETE FROM tbl WHERE id = 1;
SELECT * FROM vw_inspect_tbl_first_page;
```